In [1]:
__package__ = "jupyter"

import unittest
from faker import Faker
import uuid
import sys

sys.path.append("..")
sys.path.append("../src")
sys.path.append("../protobuf")

from index import Index
from store import Store
from sample_pb2 import Example, Type
from ipfs import Ipfs

Faker.seed(0)
fake = Faker()

class TestStore():
    def __init__(self):
        self.borrower = str(uuid.uuid4())
        self.lender = str(uuid.uuid4())
        self.loan = str(uuid.uuid4())

    def test_store_and_read_data(self):
        # create and store example data
        data = Example(type=Type.BUZZ, content="fizz")
        index = Index(
            prefix="loan",
            index={
                "borrower": self.borrower,
                "lender": self.lender
            }, subindex=Index(
                index={
                    "loan": self.loan
                }, subindex=Index(
                    index={
                        "payment": str(uuid.uuid4())
                    }
                )
            )
        )
        store = Store(index=index, writer=data)
        store.add()

        # read stored data and check equality
        store2 = Store(index=index, reader=Example())
        store2.read()
        assert store2.reader == data

    def test_query_borrower_and_lender(self):
        # create and store example data
        data = Example(type=Type.BUZZ, content="fizz")
        index = Index(
            prefix="loan",
            index={
                "borrower": self.borrower,
                "lender": self.lender
            }, subindex=Index(
                index={
                    "loan": self.loan
                }, subindex=Index(
                    index={
                        "payment": str(uuid.uuid4())
                    }
                )
            )
        )
        store = Store(index=index, writer=data)
        store.add()

        # query for data by borrower and lender
        query_index = Index(
            index={
                "borrower": self.borrower,
                "lender": self.lender
            },
            prefix="loan"
        )
        results = list(Store.query(query_index, Ipfs()))

        # check that the result matches the original data
        assert len(results) == 1
        assert results[0].index == index

    def test_query_borrower_only(self):
        # create and store example data
        data = Example(type=Type.BUZZ, content="fizz")
        index = Index(
            prefix="loan",
            index={
                "borrower": self.borrower,
                "lender": self.lender
            }, subindex=Index(
                index={
                    "loan": self.loan
                }, subindex=Index(
                    index={
                        "payment": str(uuid.uuid4())
                    }
                )
            )
        )
        store = Store(index=index, writer=data)
        store.add()

        # query for data by borrower only
        query_index = Index(
            index={
                "borrower": self.borrower
            },
            prefix="loan",
            size=2
        )
        results = list(Store.query(query_index, Ipfs()))

        # check that the result matches the original data
        assert len(results) == 1
        assert results[0].index == index


In [2]:
test = TestStore()

In [4]:
test.test_query_borrower_and_lender()

Skipped file: loan/borrower_40fcad1f-475d-4503-add0-f4dd564cca87.lender_82d3c22b-633f-4bf9-bc31-94cac5874579/Entries


AssertionError: 

In [4]:
ipfs = Ipfs()

In [5]:
example = Example(
    type = Type.FIZZ,
    content = "hello world!"
)

In [6]:
content = example.SerializeToString()

In [7]:
filename = "loan/borrower_e133c68e-9443-4acb-b16e-3411b098fcda.lender_91227598-ed2b-4900-8f6d-5b1c4c7ce960/loan_66af7f74-33ad-4d67-b229-7d2d47eeb616/payment_50ef794e-2734-4345-bcaa-c1ffa31002a2"

In [8]:
ipfs.add(filename, content)